In [22]:
class MerkleTree:
  def __init__(self, strings, hash_func):
    self.strings = strings
    self.hash_func = hash_func
  
  ## Obtener la raiz
  def get_root(self, strings = []):
    n = len(strings)
    if not strings:
      strings = self.strings
      n = len(strings)
    if len(strings)%2 > 0 and len(strings)!=1:
      ## En caso de que el nivel de hojas no sea par, se duplica la útimo
      strings.append(strings[-1])
      n = len(strings)

    
    if n > 1:
      ## Se arma recursivamente el árbol
      ## Se unen los pares de hojas
      return self.get_root([self.hash_func(strings[i]) + self.hash_func(strings[i+1]) for i in range(0, len(strings), 2)])
    elif n == 1:
      return self.hash_func(strings[0])
    

  def get_proof_for(self, item, strings = [], proof = []):
    if not strings:
      ## Este el caso inicial
      ## Cuando la función por primera vez
      ## no tiene lista de strings
      strings = self.strings
    if len(strings)%2 > 0 and len(strings)!=1:
      ## En caso de que el nivel de hojas no sea par, se duplica la útimo
      strings.append(strings[-1])

    if item not in strings:
      ## En caso de que el item no esté en la lista de strings
      ## se retorna None
      return None
    elif len(strings) == 1:
      return proof
    else:
      ## Se revisa si el item ingresado está a la derecha o izquieda del nodo padre
      ## se usa módulo 2
      i = strings.index(item)
      l = 'i' if i%2 == 0 else 'd'
      if l == 'i':
        ## si es izquierdo, los datos de la hoja derecha servirán de prueba
        near = strings[i+1]
        side = 'd'
        ## se arma el argumento que entra a la función de has del nodo padre
        new_item = self.hash_func(item) +self.hash_func(near)
      else:
        ## si es derecho, los datos de la hoja izquierda servirán de prueba
        near = strings[i-1]
        side = 'i'
        ## se arma nodo el argumento que entra a la función de has del nodo padre
        new_item = self.hash_func(near) + self.hash_func(item)
      ## agregamos la prueba
      proof.append((self.hash_func(near), side))
      ## Recursivamente buscamos al vecino del nodo padre que acabamos de armar (new_item)
      ## Como lista de strings ingresamos los nuevos argumentos de las funciones de hash
      ## del siguiente nivel del arbol
      return self.get_proof_for(new_item, [self.hash_func(strings[i]) + self.hash_func(strings[i+1]) for i in range(0, len(strings), 2)], proof)

In [24]:
def verify(root, item, proof, hash_func):
  if hash_func(item) == root:
    ## En caso de que lleguemos armar la raiz
    ## Entonces las pruebas sirven y retornamos True
    return True
  if len(proof)>0:
    ## Se va armando la raiz del árbol desde abajo hacia arriba
    if proof[0][1] == 'd':
      new_item = hash_func(item) + proof[0][0]
      return verify(root, new_item, proof[1:], hash_func)
    elif proof[0][1] == 'i':
      new_item = proof[0][0] + hash_func(item)
      return verify(root, new_item, proof[1:], hash_func)
  ## En caso de no armar correctamente la raiz
  ## se retorna False
  return False